In [ ]:
# To show the installation environment. Disabled due to git conflicts when working on 2 machines, should be changed for a minimal working installation.
# import sys
 
 
# print("User Current Version:-", sys.version)
# print()
# import pkg_resources
# installed_packages = pkg_resources.working_set
# installed_packages_list = sorted(["%s==%s" % (i.key, i.version)
#    for i in installed_packages])
# for i in installed_packages_list:
#     print(i)

# Privacy results (FILIP):
The aim of this part is to reproduce as closely as possible FILIP's results. A perfect reproduction is unlikely given the different machine setup, as I do not know on how many real machine the experiment was performed.
## Load data

In [ ]:
import pandas as pd
import numpy as np
import os
import matplotlib.pyplot as plt

# Load dynamic data
# Craft folder path

dynamic_root_dir = 'my_results/2439939_filip_dynamic/'
fc_root_dir = 'my_results/2439940_filip_fullyconnected/'
static_root_dir = 'my_results/2439941_filip_4regular_dl/'


env = "NIID-full-model-sharing-dynamic"
FILE_NAMES = ['privacy-summary-PRE-STEP.json', 'privacy-summary-POST-STEP.json']
location = FILE_NAMES[1]
MAX_PROCESSES = 24
MAX_MACHINES =  4
MAX_ITERATIONS=2500


machine_folder = 'machine{}'
privacy_folder = 'privacy'
summary_folder = 'summary'
process_folder = '{}'

def load_privacy_data(path_dir):
    data = pd.DataFrame({})  

    for location in FILE_NAMES:
        for machine in range(MAX_MACHINES):
            for rank in range(MAX_PROCESSES):
                print(f"Loading {location} for machine {machine} and rank {rank}  ",end = "\r")
                file = os.path.join(path_dir, machine_folder.format(machine),privacy_folder, summary_folder, process_folder.format(machine*MAX_PROCESSES+rank), location)
                tmp_df = pd.read_json(file)
                tmp_df = tmp_df[tmp_df.iteration < MAX_ITERATIONS]
                #tmp_df['location_of_attack']= file.split('.')[0]
                data = pd.concat([data,tmp_df])
    return data


# Load data
print(f"Loading dynamic data:")
dynamic_data = load_privacy_data(dynamic_root_dir)
print(f"\nLoading fully-connected data:")
fc_data = load_privacy_data(fc_root_dir)
print(f"\nLoading static data:")
static_data = load_privacy_data(static_root_dir)

           
# Print
dynamic_data

## Group by iterations and plot moving min,max,avg

In [ ]:
# Define plot attributes
fontsize=20
linewidth = 5
alpha = 0.2
figsize = (17,12)


metrics = ["Attacker advantage","AUC"]
metrics = [metrics[0]]
window_size = 50
linewidth = 3
fig,axs = plt.subplots(len(metrics) * 3 ,2,sharey='row',sharex= True, figsize=(figsize[0]*2,figsize[1]*len(metrics)*3))


def plot_data(data, label):
    # Group by iteration and extracted mean and std
    entire_dataset = data[data['slice feature'] == 'Entire dataset']
    for ii,metric in enumerate(metrics):
        for j,location in enumerate(data.location_of_attack.unique()):
            i = 3*ii
            print(f"{metric}, {location}, {i}")
            location_label = "".join(location.split('-'))

            # Extract relevant data
            location_data = entire_dataset[entire_dataset.location_of_attack==location]

            # Aggregate
            columns = ['iteration', 'Attacker advantage', 'AUC']
            averaged = location_data[columns].groupby('iteration').agg([np.max, np.min, np.mean, np.std])
            
            # Plot
            metric_data = averaged[metric].rolling(window=window_size).mean()
                        
            data_max, data_min = metric_data["amax"], metric_data["amin"]
            

            axs[i,j].plot(averaged.index, data_min,label=label+'-MIN', linewidth=linewidth)
            axs[i,j].set_xlabel('Iteration', fontsize=fontsize)
            axs[i,j].set_ylabel(metric, fontsize=fontsize)
            axs[i,j].set_title(f"{env}    |   {location}    |   {metric}    |  comparison    |   Min of each log")
            axs[i,j].tick_params(labelbottom=True,labelleft = True)



            axs[i+1,j].plot(averaged.index, data_max,label=label+'-MAX',linewidth=linewidth)
            axs[i+1,j].set_title(f"{env}    |   {location}    |   {metric}    |  comparison    |   Max of each log ")
            axs[i+1,j].set_ylabel(metric, fontsize=fontsize)
            axs[i+1,j].set_xlabel('Iteration', fontsize=fontsize)
            axs[i+1,j].legend(fontsize=fontsize)



            mean = metric_data['mean']
            std = metric_data['std']
            axs[i+2,j].plot(averaged.index, mean ,label= label + "-MEAN",linewidth=linewidth*2)
            axs[i+2,j].fill_between(averaged.index, mean-2*std, mean+2*std, alpha=alpha)
            axs[i+2,j].set_title(f"{env}    |   {location}    |   {metric}    |  comparison   |   Mean with min-max filled")
            axs[i+2,j].set_ylabel(metric, fontsize=fontsize)
            axs[i+2,j].set_xlabel('Iteration', fontsize=fontsize)


plot_data(dynamic_data,"DYNAMIC")
plot_data(fc_data,"FULLYCONNECTED")
plot_data(static_data,"STATIC")

        
for ii,metric in enumerate(metrics):
    for j,location in enumerate(static_data.location_of_attack.unique()):
        i = 3*ii
        axs[i,j].legend(fontsize=fontsize)
        axs[i,j].grid()
        axs[i,j].tick_params(labelbottom=True,labelleft = True)
        extent = axs[i,j].get_window_extent().transformed(fig.dpi_scale_trans.inverted())
        # axs[i,j].figure.savefig(f"NIID-min-{location}-{metric.replace(' ','_')}.jpg", bbox_inches=extent.expanded(1.2,1.2))
        
        axs[i+1,j].legend(fontsize=fontsize)
        axs[i+1,j].grid()
        axs[i+1,j].tick_params(labelbottom=True,labelleft = True)
        extent = axs[i+1,j].get_window_extent().transformed(fig.dpi_scale_trans.inverted())
        # axs[i+1,j].figure.savefig(f"NIID-max-{location}-{metric.replace(' ','_')}.jpg", bbox_inches=extent.expanded(1.2,1.2))
        
        axs[i+2,j].legend(fontsize=fontsize)
        axs[i+2,j].grid()
        axs[i+2,j].tick_params(labelbottom=True,labelleft = True)
        extent = axs[i+2,j].get_window_extent().transformed(fig.dpi_scale_trans.inverted())
        # axs[i+2,j].figure.savefig(f"NIID-mean-{location}-{metric.replace(' ','_')}.jpg", bbox_inches=extent.expanded(1.2,1.2))

# Gradient profiling

## Load data:

In [ ]:
#Config used : 
# walltime="5:00:00"
# GRAPH_FILE="36regular_4degree.edges"
# NB_MACHINE=2
# NB_PROC_PER_MACHINE=36//2
# NB_ITERATION=3000
# EVAL_FILE= ["testingPeerSampler.py", "testingPeerSamplerDynamic.py"]
# TEST_AFTER=5
# LOG_LEVEL="INFO"
# CONFIG_NAME="only_training_gaussiannoise.ini"
# CONFIG_FILE="decentralizepy/run_configuration/"+CONFIG_NAME
# cluster = "paravance"


import pandas as pd
import numpy as np
import os
import matplotlib.pyplot as plt


# Load data
# Craft folder path

gradient_profiling_dir_static = "my_results/1932209_gradient_profiling_36nodes_static/"
gradient_profiling_dir_dynamic = "my_results/1932227_gradient_profiling_36nodes_dynamic/"


MAX_PROCESSES = 18
MAX_MACHINES =  2
MAX_ITERATIONS=3000

machine_folder = 'machine{}'
result_file = '{}_results.json'


# Load data
profiling_data_static = pd.DataFrame({})
profiling_data_dynamic = pd.DataFrame({})

for machine in range(MAX_MACHINES):
    for rank in range(MAX_PROCESSES):
        print(f"Loading results for machine {machine} and rank {rank}.  ",end = "\r")

        file = os.path.join(gradient_profiling_dir_static, machine_folder.format(machine),result_file.format(rank))
        tmp_df = pd.read_json(file)
        # print(f"tmp_df size : {tmp_df.size}. profiling_data size : {profiling_data_static.size}                        ")
        profiling_data_static = pd.concat([profiling_data_static,tmp_df])
           
        file = os.path.join(gradient_profiling_dir_dynamic, machine_folder.format(machine),result_file.format(rank))
        tmp_df = pd.read_json(file)
        # print(f"tmp_df size : {tmp_df.size}. profiling_data size : {profiling_data_dynamic.size}                        ")
        profiling_data_dynamic = pd.concat([profiling_data_dynamic,tmp_df])

# Print
# profiling_data_static
profiling_data_dynamic

Regroup and average the grad_mean and grad_std. The other metrics are logged, but not important here

In [ ]:
def get_iteration_result(data):
    data_collapsed = data.groupby(level=0)
    return data_collapsed.agg({'grad_norm':['mean','std','min','max']})


iteration_avg_results_static = get_iteration_result(profiling_data_static)


iteration_avg_results_dynamic=get_iteration_result(profiling_data_dynamic)


# iteration_avg_results_static
iteration_avg_results_dynamic

In [ ]:
save_directory="assets/"
fig,axs = plt.subplots(2,1,sharex= True,figsize=(15,10))
for key,results in {"Static": iteration_avg_results_static, "Dynamic" : iteration_avg_results_dynamic}.items():
    axs[0].plot(results.index,results["grad_norm"]["mean"],label=key)
    
    axs[1].plot(results.index,results["grad_norm"]["std"], label = key)

axs[0].set_ylabel("Average gradient norm")

axs[1].set_ylabel("Gradient norm std")
axs[1].set_title("Average of gradient standard deviation value accross all processes")
axs[0].set_title("Average of gradient norm value accross all processes")
axs[0].tick_params(labelbottom=True,labelleft = True)
axs[1].tick_params(labelbottom=True,labelleft = True)
axs[1].legend()
axs[1].grid()
axs[0].legend()
axs[0].grid()
extent = axs[0].get_tightbbox(fig.canvas.get_renderer()).transformed(fig.dpi_scale_trans.inverted())
fig.savefig(f"{save_directory}/{axs[0].get_title()}", bbox_inches=extent)

extent = axs[1].get_tightbbox(fig.canvas.get_renderer()).transformed(fig.dpi_scale_trans.inverted())
fig.savefig(f"{save_directory}/{axs[1].get_title()}", bbox_inches=extent)

# Loss visualization functions
The aim of these functions will be to make it easier to visualize/compare losses between processes

In [ ]:
# target_dir_static = "my_results/1932209_gradient_profiling_36nodes_static/"
# target_dir_dynamic = "my_results/1932227_gradient_profiling_36nodes_dynamic/"

# target_dir = {
#     "Static" : "my_results/1932690_mia_36nodes_gaussiannoise_static/",
#     "Dynamic" : "my_results/1932691_mia_36nodes_gaussiannoise_dynamic/" 
# }
# target_dir_static = 'my_results/1933975_training_gaussiannoise/'

target_dir = {
    # "No noise" : "my_results/1933979_training_unnoised",
    # "Gaussian /32" : "my_results/1933975_training_gaussiannoise_32th",
    # "Gaussian /16" : "my_results/1934049_training_gaussiannoise_16th",
    # "Gaussian /8" : "my_results/1934011_training_gaussiannoise_8th",
    # "Gaussian /4" : "my_results/1934020_training_gaussiannoise_4th", 
    # "ZeroSumNoise /32" : "my_results/1933977_training_zerosumnoise_32th",
    # "ZeroSumNoise /16" : "my_results/1934050_training_zerosumnoise_16th/",
    # "ZeroSumNoise /8" : "my_results/1934012_training_zerosumnoise_8th",
    # "ZeroSumNoise /4" : "my_results/1934033_training_zerosumnoise_4th/",

    # "No noise Dynamix" : "my_results/4081377_training_nonoise_dynamic/",
    # "Gaussian /32 Dynamic" : "my_results/4081366_training_gaussiannoise_32th_dynamic/",
    # "ZeroSumNoise /32 Dynamic" : "my_results/4081367_training_zerosumnoise_32th_dynamic/",




    
    # "No noise static" : "my_results/1934908_mia_unnoised_static",
    # "No noise dynamic" : "my_results/4088192_mia_unnoised_dynamic/",
    # "Gaussian /16 static": "my_results/1934909_mia_gaussiannoise_static_16th/",
    # "ZeroSum /16 static" : "my_results/1934910_mia_zerosummnoise_static_16th/",
    # "Gaussian /16 dynamic" : "my_results/1934917_mia_gaussiannoise_dynamic_16th/",
    # "ZeroSum /16 dynamic" : "my_results/1934918_mia_zerosumnoise_dynamic_16th/",


    # "Amnesia no noise Static" : "my_results/2395524_mia_unnoised_static_amnesia/",
    # "Amnesia Gaussian /16 Static" : "my_results/2395525_mia_gaussiannoise_static_amnesia_16th",
    # "Amnesia ZeroSum /16 Static" : "my_results/2395526_mia_zerosumnoise_static_amnesia_16th",

    # "Amnesia no noise Dynamic" : "my_results/2395527_mia_unnoised_dynamic_amnesia",
    # "Amnesia Gaussian /16 Dynamic" : "my_results/2395528_mia_gaussiannoise_dynamic_amnesia_16th",
    # "Amnesia ZeroSum /16 Dynamic": "my_results/2395529_mia_zerosumnoise_dynamic_amnesia_16th"

}

target_dir = { # The new study, with more logging about noise as well as rectification for zerosum magnitude 
    "No noise static" : "my_results/1936484_static_unnoised",
    "No noise dynamic" : "my_results/1936488_dynamic_unnoised/",
    "Gaussian /16 static": "my_results/1936483_static_gaussian_16th",
    "ZeroSum /16 static" : "my_results/1936485_static_zerosum_16th/",
    "Gaussian /16 dynamic" : "my_results/1936489_dynamic_gaussian_16th/",
    "ZeroSum /16 dynamic" : "my_results/1936490_dynamic_zerosum_16th/",
}


#Needed information about how the folders are organized, can be easily extracted from just the folders.
TOTAL_PROCESSES = 36
MAX_MACHINES =  3
MAX_ITERATIONS=3000


import pandas as pd
import numpy as np
import os
import matplotlib.pyplot as plt

assert TOTAL_PROCESSES%MAX_MACHINES == 0
MAX_PROCESSES = TOTAL_PROCESSES//MAX_MACHINES

machine_folder = 'machine{}'
result_file = '{}_results.json'


# Load 
data_dict = {}
for key in target_dir:
    data_dict[key] = pd.DataFrame({})

# data_static = pd.DataFrame({})
# data_dynamic = pd.DataFrame({})
# data_fc = pd.DataFrame({})

for machine in range(MAX_MACHINES):
    for rank in range(MAX_PROCESSES):
        print(f"Loading results for machine {machine} and rank {rank}.  ",end = "\r")
        uid = rank + machine * MAX_PROCESSES

        for (key, dir) in target_dir.items():
            file = os.path.join(dir, machine_folder.format(machine), result_file.format(rank))
            tmp_df = pd.read_json(file)
            tmp_df["uid"] = uid # Manually add the uid for further processing
            data_dict[key] = pd.concat([data_dict[key],tmp_df])
           
        # file = os.path.join(target_dir_dynamic, machine_folder.format(machine), result_file.format(rank))
        # tmp_df = pd.read_json(file)
        # tmp_df["uid"] = uid # Manually add the uid for further processing
        # data_dynamic = pd.concat([data_dynamic,tmp_df])

        # file = os.path.join(target_dir_fc, machine_folder.format(machine), result_file.format(rank))
        # tmp_df = pd.read_json(file)
        # profiling_data_fc = pd.concat([profiling_data_fc,tmp_df])

# Print
# data_static
# data_dict

Drop the NaN values, and create a group by train_loss / test_loss with all the necessary metrics

In [ ]:

def get_iteration_result(data):
    data_collapsed = data.groupby(level=0)
    return data_collapsed.agg({'train_loss':['mean','std','min','max'], "test_loss":['mean','std','min','max'], "test_acc": ['mean','std','min','max']})

aggregated_data_label_dict = {}

for label,data in data_dict.items():
    print(f"Handling {label}")
    data = data[["uid","train_loss","test_loss","test_acc"]] # Since we are interested in losses
    data = data.dropna()
    data_dict[label] = data
    data_aggregated = get_iteration_result(data)
    aggregated_data_label_dict[label] = data_aggregated



# aggregated_data_label_dict["Static"]
# data_dict

We can check that the number of row is what we expect (`ITERATION_NUMBER//TEST_AFTER`, since measurement are done every `TEST_AFTER` steps)

Now, print the individual and global stats:
# Individual loss display 

In [ ]:
# Must be user defined, feel free to adjust figsize (limits to the naïve autoscaling performed here)
figsize = (10,10)
to_display = [0,1,2] # The process uid that will be shown
attributes = ["train_loss", "test_loss", "test_acc"]  # The metric we want to evaluate, must be computed in the table
alpha = 0.2


subplot_dim = (len(to_display),len(attributes))
fig,axs = plt.subplots(subplot_dim[0],subplot_dim[1],sharex= True, figsize=(figsize[0]* subplot_dim[0], figsize[1] * subplot_dim[1]))

for i,uid in enumerate(to_display):
    for j, attribute in enumerate(attributes):
        if "test" in attribute:
            axs[i][j].set_title(f"{attribute} of process {uid}, evaluated on global test set")
        elif "train" in attribute: 
            axs[i][j].set_title(f"{attribute} of process {uid}, evaluated on local train set") 
        else:
            axs[i][j].set_title(f"{attribute} of {uid}")
        axs[i][j].set_ylabel(f"{attribute} of process {uid}")


for key,data in data_dict.items(): 
    for i,uid in enumerate(to_display):
        data_to_plot = data[data["uid"] == uid] 
        for j, attribute in enumerate(attributes):
            axs[i][j].plot(data_to_plot.index,data_to_plot[attribute],label=key)

for i in range(subplot_dim[0]):
    for j in range(subplot_dim[1]):
        axs[i][j].legend()
        axs[i][j].grid()

# Global loss display

In [ ]:
# Must be user defined, feel free to adjust figsize (limits to the naïve autoscaling performed here)
figsize = (10,10)
attributes = ["train_loss", "test_loss", "test_acc"]
metrics = ["mean", "std"]  # The metric we want to evaluate, must be computed in the table
alpha = 0.2



subplot_dim = (len(attributes),len(metrics))
fig,axs = plt.subplots(subplot_dim[0],subplot_dim[1],sharex= True, figsize=(figsize[0]* subplot_dim[0], figsize[1] * subplot_dim[1]))

# Loop to set axis and sublpot titles. 
for i,attribute in enumerate(attributes):
    for j, metric in enumerate(metrics):
        if "test" in attribute:
            axs[i][j].set_title(f"{metric} of {attribute}, evaluated on global test set")
        elif "train" in attribute: 
            axs[i][j].set_title(f"{metric} of {attribute}, evaluated on local train set") 
        else:
            axs[i][j].set_title(f"{metric} of {attribute}")
        axs[i][j].set_ylabel(f"{attribute} {metric}")

#Plots the data
for key,data in aggregated_data_label_dict.items(): 
    for i, attribute in enumerate(attributes):
        for j, metric in enumerate(metrics):
            axs[i][j].plot(data.index,data[attribute][metric],label=key)  
            if metric=="mean": # When displaying the mean, we also display the min and max for each iteration. 
                axs[i][j].fill_between(data.index, data[attribute]["min"], data[attribute]["max"], alpha=alpha)
   

for i in range(subplot_dim[0]):
    for j in range(subplot_dim[1]):
        axs[i][j].legend()
        axs[i][j].grid()

## Noise control:

In [ ]:
data_dict[]

# Gaussian noise experiments

In [ ]:
import pandas as pd
import numpy as np
import os
import matplotlib.pyplot as plt

# Load dynamic data
# Craft folder path

# static_root_dir = 'my_results/1932690_mia_36nodes_gaussiannoise_static/'
# dynamic_root_dir = 'my_results/1932691_mia_36nodes_gaussiannoise_dynamic/'
# # fc_root_dir = 'my_results/2439940_filip_fullyconnected/'

#target_dir = {
    # "No noise static" : "my_results/1934908_mia_unnoised_static",
    # "No noise dynamic" : "my_results/4088192_mia_unnoised_dynamic/",
    # "Gaussian /16 static": "my_results/1934909_mia_gaussiannoise_static_16th/",
    # "ZeroSum /16 static" : "my_results/1934910_mia_zerosummnoise_static_16th/",
    # "Gaussian /16 dynamic" : "my_results/1934917_mia_gaussiannoise_dynamic_16th/",
    # "ZeroSum /16 dynamic" : "my_results/1934918_mia_zerosumnoise_dynamic_16th/",


    # "Amnesia no noise static" : "my_results/2395524_mia_unnoised_static_amnesia/",
    # "Amnesia Gaussian /16 static" : "my_results/2395525_mia_gaussiannoise_static_amnesia_16th",
    # "Amnesia ZeroSum /16 static" : "my_results/2395526_mia_zerosumnoise_static_amnesia_16th",

    # "Amnesia no noise dynamic" : "my_results/2395527_mia_unnoised_dynamic_amnesia",
    # "Amnesia Gaussian /16 dynamic" : "my_results/2395528_mia_gaussiannoise_dynamic_amnesia_16th",
    # "Amnesia ZeroSum /16 dynamic": "my_results/2395529_mia_zerosumnoise_dynamic_amnesia_16th"

#}

target_dir = { # The new study, with more logging about noise as well as rectification for zerosum magnitude 
    "No noise static" : "my_results/1936484_static_unnoised",
    "No noise dynamic" : "my_results/1936488_dynamic_unnoised/",
    "Gaussian /16 static": "my_results/1936483_static_gaussian_16th",
    "ZeroSum /16 static" : "my_results/1936485_static_zerosum_16th/",
    "Gaussian /16 dynamic" : "my_results/1936489_dynamic_gaussian_16th/",
    "ZeroSum /16 dynamic" : "my_results/1936490_dynamic_zerosum_16th/",
}



env = "NIID-full-model-sharing-dynamic"
LOCATIONS_OF_ATTACKS = ["PRE-STEP", "POST-STEP"]
TOTAL_PROCESSES = 36
MAX_MACHINES =  3
MAX_ITERATIONS=3000




assert TOTAL_PROCESSES%MAX_MACHINES == 0
MAX_PROCESSES = TOTAL_PROCESSES//MAX_MACHINES
FILE_NAMES = [f"privacy-summary-{loc}.json" for loc in LOCATIONS_OF_ATTACKS]
machine_folder = 'machine{}'
privacy_folder = 'privacy'
summary_folder = 'summary'
process_folder = '{}'

def load_privacy_data(path_dir):
    data = pd.DataFrame({})  

    for location in FILE_NAMES:
        for machine in range(MAX_MACHINES):
            for rank in range(MAX_PROCESSES):
                print(f"Loading {location} for machine {machine} and rank {rank}  ",end = "\r")
                file = os.path.join(path_dir, machine_folder.format(machine),privacy_folder, summary_folder, process_folder.format(machine*MAX_PROCESSES+rank), location)
                tmp_df = pd.read_json(file)
                tmp_df = tmp_df[tmp_df.iteration < MAX_ITERATIONS]
                #tmp_df['location_of_attack']= file.split('.')[0]
                data = pd.concat([data,tmp_df])
    return data

data_dict = {}
for key,dir in target_dir.items():
    print(f"Loading privacy data at \"{dir}\"")
    data_dict[key] = load_privacy_data(dir)

# # Load data
# print(f"Loading dynamic data:\t\t")
# dynamic_data = load_privacy_data(dynamic_root_dir)
# print(f"Loading static data:\t\t")
# static_data = load_privacy_data(static_root_dir)
# # print(f"Loading fully-connected data:\t\t")
# # fc_data = load_privacy_data(fc_root_dir)


In [ ]:
# Define plot attributes
fontsize=20
linewidth = 5
alpha = 0.1
figsize = (17,12)


metrics = ["Attacker advantage","AUC"]
metrics = [metrics[0]]
window_size = 50
linewidth = 3
fig,axs = plt.subplots(len(metrics) * 3 ,2,sharey='row',sharex= True, figsize=(figsize[0]*2,figsize[1]*len(metrics)*3))


def plot_data(data, label):
    # Group by iteration and extracted mean and std
    line_style = '-'
    if "Amnesia" in label:
        line_style =  "--"
    entire_dataset = data[data['slice feature'] == 'Entire dataset']
    for ii,metric in enumerate(metrics):
        for j,location in enumerate(data.location_of_attack.unique()):
            i = 3*ii
            print(f"{metric}, {location}, {i}")
            location_label = "".join(location.split('-'))

            # Extract relevant data
            location_data = entire_dataset[entire_dataset.location_of_attack==location]

            # Aggregate
            columns = ['iteration', 'Attacker advantage', 'AUC']
            averaged = location_data[columns].groupby('iteration').agg([np.max, np.min, np.mean, np.std])
            
            # Plot
            metric_data = averaged[metric].rolling(window=window_size).mean()
                        
            data_max, data_min = metric_data["amax"], metric_data["amin"]
            

            axs[i,j].plot(averaged.index, data_min, line_style, label=label+'-MIN', linewidth=linewidth)
            axs[i,j].set_xlabel('Iteration', fontsize=fontsize)
            axs[i,j].set_ylabel(metric, fontsize=fontsize)
            axs[i,j].set_title(f"{env}    |   {location}    |   {metric}    |  comparison    |   Min of each log")
            axs[i,j].tick_params(labelbottom=True,labelleft = True)


            axs[i+1,j].plot(averaged.index, data_max, line_style,label=label+'-MAX',linewidth=linewidth)
            axs[i+1,j].set_title(f"{env}    |   {location}    |   {metric}    |  comparison    |   Max of each log ")
            axs[i+1,j].set_ylabel(metric, fontsize=fontsize)
            axs[i+1,j].set_xlabel('Iteration', fontsize=fontsize)
            axs[i+1,j].legend(fontsize=fontsize)


            mean = metric_data['mean']
            std = metric_data['std']
            axs[i+2,j].plot(averaged.index, mean, line_style, label= label + "-MEAN",linewidth=linewidth*2)
            axs[i+2,j].fill_between(averaged.index, mean-2*std, mean+2*std, alpha=alpha)
            axs[i+2,j].set_title(f"{env}    |   {location}    |   {metric}    |  comparison   |   Mean with min-max filled")
            axs[i+2,j].set_ylabel(metric, fontsize=fontsize)
            axs[i+2,j].set_xlabel('Iteration', fontsize=fontsize)
            
for key,data in data_dict.items(): 
    plot_data(data,key)    
# plot_data(dynamic_data,"DYNAMIC")
# plot_data(fc_data,"FULLYCONNECTED")
# plot_data(static_data,"STATIC")

        
for ii,metric in enumerate(metrics):
    for j,location in enumerate(LOCATIONS_OF_ATTACKS):
        i = 3*ii
        axs[i,j].legend(fontsize=fontsize)
        axs[i,j].tick_params(labelbottom=True,labelleft = True)
        extent = axs[i,j].get_window_extent().transformed(fig.dpi_scale_trans.inverted())
        axs[i,j].grid()
        # axs[i,j].figure.savefig(f"NIID-min-{location}-{metric.replace(' ','_')}.jpg", bbox_inches=extent.expanded(1.2,1.2))
        
        axs[i+1,j].legend(fontsize=fontsize)
        axs[i+1,j].tick_params(labelbottom=True,labelleft = True)
        extent = axs[i+1,j].get_window_extent().transformed(fig.dpi_scale_trans.inverted())
        axs[i+1,j].grid()
        # axs[i+1,j].figure.savefig(f"NIID-max-{location}-{metric.replace(' ','_')}.jpg", bbox_inches=extent.expanded(1.2,1.2))

        axs[i+2,j].legend(fontsize=fontsize)
        axs[i+2,j].tick_params(labelbottom=True,labelleft = True)
        extent = axs[i+2,j].get_window_extent().transformed(fig.dpi_scale_trans.inverted())
        axs[i+2,j].grid()
        # axs[i+2,j].figure.savefig(f"NIID-mean-{location}-{metric.replace(' ','_')}.jpg", bbox_inches=extent.expanded(1.2,1.2))

